### 数据处理
- 原始数据集直接从Kaggle上下载，解压后train目录下一共有25000张图片，test目录下一共有12500张图片
- 我们需要通过Keras ImageDataGenerator的flow_from_directory方法来加载我们的图片，所以我们需要将训练集和测试集的图片放到子文件夹中
- 测试集很简单，建一个子目录，将文件全部移动过去就行
- 训练集需要建立两个子目录，将狗和猫的图片分别移动到两个子目录中去
- 训练集中的图片名称格式为{种类}.序号.jpg，比如 cat.1.jpg 或 dog.1.jpg。我们可以利用命名规则来移动训练集的图片

当前的图片目录结构如下
```
data
 ├── test   [12500 images]
 └── train  [25000 images]
```
预处理后图片目录结构如下
```
data
 ├── test
 │   └── none [12500 images]
 └── train
     ├── cat  [12500 images]
     └── dog  [12500 images]
```

In [10]:
# 数据处理代码
import os
from shutil import move

train_src = 'data/train/'
test_src = 'data/test/'
dog_dest = 'data/train/dog/'
cat_dest = 'data/train/cat/'
test_dest = 'data/test/none/'

#创建子目录
os.makedirs(dog_dest, exist_ok=True)
os.makedirs(cat_dest, exist_ok=True)
os.makedirs(test_dest, exist_ok=True)

#移动测试图片至子文件夹中
for root, dirs, files in os.walk(test_src):
    if(root == test_src):
        for name in files:
            if(name.find('jpg')>-1):
                move(test_src + name, test_dest + name)

#移动训练集图片至对应的子文件夹中
for root, dirs, files in os.walk(train_src):
    if(root == train_src):
        for name in files:
            if(name.find('jpg')>-1 and name.find('cat')>-1):
                move(train_src + name, cat_dest + name)
            elif(name.find('jpg')>-1 and name.find('dog')>-1):
                move(train_src + name, dog_dest + name)
            else:
                pass

print("cat images: ", 
      len([name for name in os.listdir(cat_dest) if os.path.isfile(os.path.join(cat_dest, name))]))
print("dog images: ", 
      len([name for name in os.listdir(dog_dest) if os.path.isfile(os.path.join(dog_dest, name))]))
print("test images: ", 
      len([name for name in os.listdir(test_dest) if os.path.isfile(os.path.join(test_dest, name))]))

cat images:  12500
dog images:  12500
test images:  12500


### 导出深度特征
- 根据当前训练以及测试集导出VGG16,VGG19,ResNet50,Xception以及InceptionV3的深度特征
- VGG16,VGG19,ResNet50要求的图片的大小为（224， 224）
- Xception，Inception要求的图片大小为（299，299）
- 先对所有数据进行一个预处理的操作，把数据缩放到-1到1之间
- 其次我们加入一个平局池化操作，一方面是缩小我们导出的深度特征文件的大小，另一方是防止过拟合
- 最后使用Keras的ImageGenerator导出深度特征的数组，存放在本地磁盘上供接下来的模型训练使用

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import time
import h5py
import math

train_data_path = 'data/train/'
test_data_path = 'data/test/'

def save_bottleneck_features(MODEL, image_size, module_name, preprocess):
    
    start_time = time.time()
    
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = Lambda(preprocess)(input_tensor)
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory(train_data_path, image_size, shuffle=False)
    test_generator = gen.flow_from_directory(test_data_path, image_size, shuffle=False, class_mode=None)

    train = model.predict_generator(train_generator)
    test = model.predict_generator(test_generator)
    
    with h5py.File("bottleneck_features/{}_bottleneck_features.h5".format(module_name)) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)
        
    end_time = time.time()
    
    print("{} extrac features total consumed: {} seconds".format(module_name, end_time - start_time))

c:\users\tracy\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
save_bottleneck_features(VGG16, (224, 224), 'VGG16', vgg16.preprocess_input)

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
VGG16 extrac features total consumed: 205.36566758155823 seconds


In [2]:
save_bottleneck_features(VGG19, (224, 224), 'VGG19', vgg19.preprocess_input)

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
VGG19 extrac features total consumed: 232.02474784851074 seconds


In [3]:
save_bottleneck_features(ResNet50, (224, 224), 'ResNet50', resnet50.preprocess_input)

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
ResNet50 extrac features total consumed: 210.49121832847595 seconds


In [4]:
save_bottleneck_features(InceptionV3, (299, 299), 'InceptionV3', inception_v3.preprocess_input)

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
InceptionV3 extrac features total consumed: 274.9980471134186 seconds


In [5]:
save_bottleneck_features(Xception, (299, 299), 'Xception', xception.preprocess_input)

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
Xception extrac features total consumed: 416.50056076049805 seconds


### 迁移学习
- 至此针对以上模型的深度特征提取完毕
- VGG16耗时约4分19秒，VGG19耗时越5分3秒，ResNet50耗时约4分14秒，InceptionV3耗时约5分45秒，Xception耗时约8分钟19秒
- 依据这些深度特征我们可以来构建新的模型并且只需要构建最后一层
- 这里首先构建一个dropout层，参数为0.5，最后构建一个全连接层来做分类

In [6]:
import h5py
import numpy as np
import pandas as pd
from keras.models import *
from keras.layers import *
from keras.preprocessing.image import *
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
import time

def retrieve_features(files):
    X_train = []
    X_test = []
    y_train = []
    
    for filename in files:
        with h5py.File(filename, 'r') as h:
            X_train.append(np.array(h['train']))
            X_test.append(np.array(h['test']))
            y_train = np.array(h['label'])
        
    X_train = np.concatenate(X_train, axis=1)
    X_test = np.concatenate(X_test, axis=1)
    
    return X_train, X_test, y_train

    

def train_model(X_train, y_train):
    # construct model
    input_tensor = Input(X_train.shape[1:])
    x = BatchNormalization()(input_tensor)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(input_tensor, x)
    # compile model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # train model
    start_time = time.time()
    model.fit(X_train, y_train, batch_size=200, epochs=10, validation_split=0.2, verbose=1)
    end_time = time.time()
    print("Trainning model total consumed:{} seconds".format(end_time - start_time))
    
    return model

def generate_submission_csv(X_test, model):

    y_pred = model.predict(X_test, verbose=1)
    y_pred = y_pred.clip(min=0.005, max=1)

    df = pd.read_csv("data/sample_submission.csv")

    gen = ImageDataGenerator()
    test_generator = gen.flow_from_directory('data/test/', (224, 224), shuffle=False, batch_size=32, class_mode=None)

    for i, fname in enumerate(test_generator.filenames):
        index = int(fname[fname.rfind('\\')+1:fname.rfind('.')])
        df.at[index-1, 'label'] = y_pred[i]

    df.to_csv('data/pred.csv', index=None)
    df.head(10)

In [9]:
# train using all the models bottleneck_features
bottleneck_files = ["bottleneck_features/VGG16_bottleneck_features.h5",
                    "bottleneck_features/VGG19_bottleneck_features.h5", 
                    "bottleneck_features/ResNet50_bottleneck_features.h5",
                    "bottleneck_features/InceptionV3_bottleneck_features.h5",
                    "bottleneck_features/Xception_bottleneck_features.h5"]


X_train, X_test, y_train = retrieve_features(bottleneck_files)

model = train_model(X_train, y_train)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 3s 138us/step - loss: 0.0430 - acc: 0.9838 - val_loss: 0.0169 - val_acc: 0.9940
Epoch 2/10
20000/20000 [==============================] - 1s 61us/step - loss: 0.0135 - acc: 0.9958 - val_loss: 0.0200 - val_acc: 0.9930
Epoch 3/10
20000/20000 [==============================] - 1s 67us/step - loss: 0.0072 - acc: 0.9975 - val_loss: 0.0329 - val_acc: 0.9900
Epoch 4/10
20000/20000 [==============================] - 1s 68us/step - loss: 0.0052 - acc: 0.9985 - val_loss: 0.0282 - val_acc: 0.9920
Epoch 5/10
20000/20000 [==============================] - 1s 67us/step - loss: 0.0037 - acc: 0.9989 - val_loss: 0.0173 - val_acc: 0.9948
Epoch 6/10
20000/20000 [==============================] - 1s 60us/step - loss: 0.0021 - acc: 0.9996 - val_loss: 0.0239 - val_acc: 0.9934
Epoch 7/10
20000/20000 [==============================] - 1s 72us/step - loss: 0.0015 - acc: 0.9998 - val_loss: 0.0375 - val_acc

In [10]:
generate_submission_csv(X_test, model)

12500/12500 [==============================] - 1s 96us/step
Found 12500 images belonging to 1 classes.


In [15]:
import math

def logloss(true_label, predicted):
    if true_label == 1:
        return -math.log(predicted)
    else:
        return -math.log(1 - predicted)

In [21]:
# 当我们预测正确时 true label = 1 , 我们预测的值为 0.999时
print(logloss(1, 0.999))
# 当我们预测正确时 true label = 0 , 我们预测的值为 0.001时
print(logloss(0, 0.001))

# 当我们预测错误时 true label = 1 , 我们预测的值为 0.4时
print(logloss(1, 0.000000000000000000001))
# 当我们预测错误时 true label = 0 , 我们预测的值为 0.6时
print(logloss(0, 0.6))

0.0010005003335835344
0.0010005003335835344
48.35428695287496
0.916290731874155
